Sentiment Analysis with BERT neural network

Basically here this model scraps the data from web with some reviews and analysis and the scores the review from 1 to 5.

In [3]:
#importing libraries
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

In [4]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [21]:
r = requests.get("https://www.yelp.com/biz/dumpling-home-san-francisco?osq=Restaurants") # requesting the data from website
soup = BeautifulSoup(r.text, "html.parser")
regex = re.compile('.*comment.*') #choosing the class with the string comment
results = soup.find_all('p',{'class':regex}) #finding the class
reviews = [result.text for result in results] #changing the results to only text format

In [22]:
r

<Response [200]>

In [23]:
soup

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/no-js/,"js");</script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en-US" http-equiv="Content-Language"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><link content="#FF1A1A" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/><link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico" rel="shortcut icon"/><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>
            window.yelp = window.yelp || {};
      

In [24]:
reviews

['Amazing Seating is limited as this is a small establishment. Serving staff seemed overwhelmed and never stopped by to refill water or check on us after we got our food but the food...',
 "Truly delicious every time, can't go wrong with anything on the menu!!! I always enjoy the green beans and any assortment of their dumplings",
 "Food 9/10Definitely get the 生煎包 aka pan-fried dumplings. The pork ones are the OG, and they literally tasted exactly like home. I really missed Shanghai after this meal, but the meal itself was super heartwarming and made me super full. Definitely ask for chili oil to pair with them. We also got the pork and crab soup dumplings, but I think those could be skipped. The flavor of the soup inside them just wasn't even close to the pan-fried ones, so that left me a bit disappointed. The green beans are fine and a good way to balance the copious amounts of dumplings we were inhaling.Atmosphere 8/10A normal Chinese restaurant I guessService 8/10Very sad we arrive

In [15]:
#Creating dataframe
df = pd.DataFrame(np.array(reviews), columns=["review"])

In [16]:
#Function to score the reviewsm, basically tokens variable takes the review and converts to numeric value
def sentiment_score(review):
  tokens = tokenizer.encode(review,return_tensors = 'pt')
  result = model(tokens)
  return int(torch.argmax(result.logits)) +  1

In [18]:
#Applying lambda to data so function works in each review
f['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [19]:
#sentiment answer will be range from 1 to 5
df

,review,sentiment
0,Amazing Seating is limited as this is a small ...,4
1,"Truly delicious every time, can't go wrong wit...",5
2,Food 9/10Definitely get the 生煎包 aka pan-fried ...,4
3,me and my friend went here today for a sunday ...,5
4,Good amount of outside-dining space. Although ...,4
5,Yum! We visited with a group of 7 and waited a...,4
6,How have I not written a review for my favorit...,1
7,"Normally a mob scene, it's manageable on Monda...",4
8,"Dumpling Home is one of my top spots in SF, an...",5
9,This is the place we bring people to when they...,5


In [20]:
df.shape

(10, 2)